In [ ]:
import oracledb
import pandas
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Retrieve CWL and SNUM from environment variables
cwl = os.getenv('CWL')
snum = os.getenv('SNUM')

In [15]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{cwl}", password=f"a{snum}", dsn=dsn)

cur = connection.cursor()

### Research Question 1: Has the adoption of digital healthcare in the United States increased from 2020 to 2024?

In [8]:
cur = connection.cursor()

query = """
SELECT 
    "YEAR",
    SUM("TOTAL_TELEHEALTH_USERS") AS total_users
FROM telehealth
WHERE "YEAR" BETWEEN 2020 AND 2024
GROUP BY "YEAR"
ORDER BY "YEAR"
"""

cur.execute(query)

# Print the results
for row in cur:
    print(row)

cur.close()


(2020, 337835176)
(2021, 233349370)
(2022, 193374238)
(2023, 151873571)


### Research Question 2: Did the use of digital healthcare services have an impact on mortality rates during the years 2020-2022?

In [9]:
cur = connection.cursor()

query2 = """
SELECT 
    M."YEAR",
    M."SEX",
    T.TOTAL_TELEHEALTH_USERS,
    M.TOTAL_DEATHS,
    ROUND(M.TOTAL_DEATHS * 1000.0 / NULLIF(T.TOTAL_TELEHEALTH_USERS, 0), 2) AS DEATHS_PER_1000_USERS
FROM (
    SELECT 
        "YEAR",
        "SEX",
        SUM("DEATHS") AS TOTAL_DEATHS
    FROM MORTALITY
    WHERE "SEX" IN ('Male', 'Female')
    GROUP BY "YEAR", "SEX"
) M
JOIN (
    SELECT 
        "YEAR",
        "SEX",
        SUM("TOTAL_TELEHEALTH_USERS") AS TOTAL_TELEHEALTH_USERS
    FROM TELEHEALTH
    WHERE "SEX" IN ('Male', 'Female') AND "AGE_GROUP" = 'All'
    GROUP BY "YEAR", "SEX"
) T
ON M."YEAR" = T."YEAR" AND M."SEX" = T."SEX"
WHERE M."YEAR" BETWEEN 2020 AND 2022
ORDER BY M."YEAR", M."SEX"
"""
cur.execute(query2)
for row in cur:
    print(row)

cur.close()

(2020, 'Female', 34300779, 1613808, 47.05)
(2020, 'Male', 24560313, 1769805, 72.06)
(2021, 'Female', 24102944, 1626100, 67.46)
(2021, 'Male', 16559408, 1838038, 111)
(2022, 'Female', 20024307, 1560591, 77.93)
(2022, 'Male', 13680073, 1719250, 125.68)


### Research Question 3: Which groups of people used digital healthcare services the most between age and race in 2020 to 2024 in the United States?

In [12]:
cur = connection.cursor()

query = """
SELECT 
    "AGE_GROUP",
    "RACE",
    SUM("TOTAL_TELEHEALTH_USERS") AS TOTAL_USERS
FROM TELEHEALTH
WHERE "YEAR" BETWEEN 2020 AND 2024
GROUP BY "AGE_GROUP", "RACE"
ORDER BY TOTAL_USERS DESC
"""
cur.execute(query)
for row in cur:
    print(row)



cur.close()

('All', 'All', 523086172)
('All', 'Non-Hispanic White', 187820339)
('65-74', 'All', 52431619)
('75-84', 'All', 34417462)
('All', 'Black/African American', 20248821)
('0-64', 'All', 19082459)
('All', 'Hispanic', 15804989)
('85 and over', 'All', 14815240)
('65-74', 'Non-Hispanic White', 13624749)
('75-84', 'Non-Hispanic White', 9426305)
('All', 'Asian/Pacific Islander', 8382634)
('0-64', 'Non-Hispanic White', 4218416)
('85 and over', 'Non-Hispanic White', 4040974)
('All', 'American Indian/Alaska Native', 1383290)
('65-74', 'Black/African American', 1317584)
('0-64', 'Black/African American', 1070002)
('65-74', 'Hispanic', 1046489)
('75-84', 'Black/African American', 696582)
('0-64', 'Hispanic', 687639)
('75-84', 'Hispanic', 610660)
('65-74', 'Asian/Pacific Islander', 601133)
('75-84', 'Asian/Pacific Islander', 431149)
('85 and over', 'Hispanic', 297051)
('85 and over', 'Black/African American', 294025)
('85 and over', 'Asian/Pacific Islander', 230250)
('0-64', 'Asian/Pacific Islander', 1